Bab 14, "Deep Computer Vision Menggunakan Convolutional Neural Networks," membahas secara mendalam tentang Convolutional Neural Networks (CNN) dan aplikasinya dalam visi komputer.


* **Pengantar CNN:** CNN muncul dari studi korteks visual otak dan telah digunakan dalam pengenalan gambar sejak tahun 1980-an. Berkat peningkatan daya komputasi dan data pelatihan, CNN telah mencapai kinerja *superhuman* dalam tugas visual yang kompleks, seperti layanan pencarian gambar, mobil swakemudi, dan sistem klasifikasi video otomatis. CNN juga digunakan dalam pengenalan suara dan pemrosesan bahasa alami.
* **Arsitektur Korteks Visual:** Penelitian David H. Hubel dan Torsten Wiesel pada tahun 1958 dan 1959 menunjukkan bahwa neuron di korteks visual memiliki *receptive field* lokal kecil, hanya bereaksi terhadap stimuli visual di area terbatas. *Receptive field* neuron yang berbeda dapat tumpang tindih dan bersama-sama menutupi seluruh bidang visual. Beberapa neuron bereaksi terhadap garis horizontal, sementara yang lain bereaksi terhadap garis dengan orientasi berbeda. Neuron tingkat yang lebih tinggi bereaksi terhadap pola yang lebih kompleks, kombinasi dari pola tingkat rendah. Pengamatan ini mengarah pada ide bahwa neuron tingkat yang lebih tinggi didasarkan pada *output* neuron tingkat rendah yang berdekatan. Arsitektur ini memungkinkan deteksi pola kompleks di seluruh bidang visual.
* **Lapisan Konvolusional (Convolutional Layers):** Ini adalah *building block* terpenting dari CNN. Neuron dalam lapisan konvolusional pertama hanya terhubung ke piksel di *receptive field* mereka, bukan ke setiap piksel di gambar masukan. Neuron di lapisan konvolusional kedua hanya terhubung ke neuron dalam persegi kecil di lapisan pertama. Arsitektur ini memungkinkan jaringan untuk fokus pada fitur tingkat rendah yang kecil di lapisan tersembunyi pertama, kemudian menggabungkannya menjadi fitur tingkat tinggi yang lebih besar di lapisan tersembunyi berikutnya. Struktur hierarkis ini umum dalam gambar dunia nyata, itulah mengapa CNN bekerja sangat baik untuk pengenalan gambar. Setiap lapisan dalam CNN direpresentasikan dalam 2D, memudahkan pencocokan neuron dengan masukan yang sesuai.
    * **Filters (Convolution Kernels):** Bobot neuron dapat direpresentasikan sebagai gambar kecil seukuran *receptive field*. Filter ini, jika diterapkan pada gambar, menghasilkan *feature map* yang menyoroti area yang paling mengaktifkan filter tersebut. Selama pelatihan, lapisan konvolusional akan secara otomatis mempelajari filter yang paling berguna.
    * **Stacking Multiple Feature Maps:** Lapisan konvolusional memiliki banyak filter dan menghasilkan satu *feature map* per filter, sehingga lebih akurat direpresentasikan dalam 3D. Semua neuron dalam *feature map* tertentu berbagi parameter yang sama (bobot dan *bias*), yang secara drastis mengurangi jumlah parameter model. Ini memungkinkan CNN mengenali pola di lokasi mana pun, tidak seperti DNN reguler yang hanya mengenali pola di lokasi tertentu. Gambar masukan juga terdiri dari beberapa *sublayer* (misalnya, RGB).
    * **TensorFlow Implementation:** Gambar masukan di TensorFlow direpresentasikan sebagai tensor 3D ($[height, width, channels]$) atau 4D untuk *mini-batch* ($[mini-batch size, height, width, channels]$). Bobot lapisan konvolusional adalah tensor 4D ($[f_h, f_w, f_{n'}, f_n]$) dan *bias* adalah tensor 1D ($[f_n]$). Fungsi `tf.nn.conv2d()` digunakan untuk menerapkan filter, dengan parameter untuk *strides* dan *padding* ("SAME" atau "VALID"). `keras.layers.Conv2D` adalah cara yang lebih mudah untuk membuat lapisan konvolusional.
    * **Memory Requirements:** Lapisan konvolusional membutuhkan banyak RAM, terutama selama pelatihan karena *reverse pass* dari *backpropagation* memerlukan semua nilai perantara yang dihitung selama *forward pass*. Untuk mengatasi kekurangan memori, dapat dicoba mengurangi ukuran *mini-batch*, mengurangi dimensi menggunakan *stride*, menghapus beberapa lapisan, menggunakan *float* 16-bit, atau mendistribusikan CNN di beberapa perangkat.
* **Lapisan Pooling (Pooling Layers):** Tujuan utamanya adalah untuk *subsample* (menyusutkan) gambar masukan guna mengurangi beban komputasi, penggunaan memori, dan jumlah parameter (membatasi risiko *overfitting*). Neuron *pooling* tidak memiliki bobot; mereka hanya menggabungkan masukan menggunakan fungsi agregasi seperti *max* atau *mean*.
    * **Max Pooling:** Jenis *pooling* yang paling umum, hanya nilai masukan maksimum dalam setiap *receptive field* yang diteruskan ke lapisan berikutnya. Max pooling memperkenalkan tingkat invarian terhadap translasi kecil, rotasi, dan skala. Namun, max pooling sangat destruktif, menghilangkan 75% nilai masukan dengan kernel $2\times2$ dan *stride* 2. Untuk tugas seperti segmentasi semantik, equivariance (perubahan kecil pada masukan menyebabkan perubahan kecil yang sesuai pada *output*) lebih diinginkan daripada invariance.
    * **Average Pooling:** Bekerja seperti max pooling tetapi menghitung rata-rata alih-alih maksimum. Max pooling umumnya berkinerja lebih baik karena mempertahankan fitur terkuat dan memberikan sinyal yang lebih bersih ke lapisan berikutnya, serta menawarkan invarian translasi yang lebih kuat dan membutuhkan komputasi yang sedikit lebih sedikit.
    * **Depthwise Max Pooling:** Dapat dilakukan di sepanjang dimensi kedalaman, bukan dimensi spasial, memungkinkan CNN mempelajari invarian terhadap berbagai fitur (misalnya, rotasi, ketebalan, kecerahan, kemiringan, warna).
    * **Global Average Pooling:** Menghitung rata-rata seluruh *feature map*, menghasilkan satu angka per *feature map* dan per *instance*. Ini berguna sebagai lapisan *output* di akhir CNN untuk tugas klasifikasi, mengurangi jumlah parameter secara signifikan dan membatasi risiko *overfitting*.
    * **TensorFlow Implementation:** `keras.layers.MaxPool2D` dan `keras.layers.AvgPool2D` untuk max dan average pooling. `tf.nn.max_pool()` untuk depthwise max pooling. `keras.layers.GlobalAvgPool2D` untuk global average pooling.
* **Arsitektur CNN:** Arsitektur CNN tipikal menumpuk beberapa lapisan konvolusional (biasanya diikuti oleh lapisan ReLU), lalu lapisan *pooling*, lalu lapisan konvolusional lainnya (+ReLU), lalu lapisan *pooling* lainnya, dan seterusnya. Gambar menjadi semakin kecil saat melalui jaringan, tetapi juga menjadi semakin dalam (dengan lebih banyak *feature map*). Di bagian atas tumpukan, ditambahkan jaringan saraf *feedforward* reguler yang terdiri dari beberapa lapisan *fully connected* (+ReLUs), dan lapisan terakhir menghasilkan prediksi.
    * **LeNet-5 (1998):** Arsitektur CNN yang paling dikenal, digunakan untuk pengenalan digit tulisan tangan. Terdiri dari lapisan konvolusional dan *pooling* dengan fungsi aktivasi `tanh` dan lapisan *fully connected* di akhir.
    * **AlexNet (2012):** Memenangkan tantangan ImageNet ILSVRC 2012 dengan margin besar. Lebih besar dan lebih dalam dari LeNet-5, dan yang pertama menumpuk lapisan konvolusional secara langsung satu sama lain. Menggunakan dropout dan *data augmentation* untuk mengurangi *overfitting*.
        * **Data Augmentation:** Secara artifisial meningkatkan ukuran *training set* dengan menghasilkan banyak varian realistis dari setiap *instance* pelatihan. Ini mengurangi *overfitting*.
        * **Local Response Normalization (LRN):** Digunakan di AlexNet untuk mendorong *feature map* yang berbeda untuk berspesialisasi dan meningkatkan generalisasi.
    * **GoogLeNet (2014):** Memenangkan tantangan ILSVRC 2014, jauh lebih dalam dari CNN sebelumnya berkat *inception modules*. GoogLeNet memiliki 10 kali lebih sedikit parameter daripada AlexNet.
        * **Inception Modules:** Memungkinkan GoogLeNet menggunakan parameter dengan lebih efisien. Modul ini menyalurkan masukan ke beberapa lapisan konvolusional dengan ukuran kernel yang berbeda ($1\times1, 3\times3, 5\times5$) dan lapisan *max pooling*, yang keluarannya kemudian digabungkan (*concatenated*). Lapisan $1\times1$ berfungsi sebagai *bottleneck layers* untuk mengurangi dimensi dan menangkap pola di sepanjang dimensi kedalaman.
    * **VGGNet (2014 Runner-up):** Arsitektur yang sederhana dan klasik dengan 2 atau 3 lapisan konvolusional dan satu lapisan *pooling* yang diulang, diikuti oleh jaringan dense di akhir. Hanya menggunakan filter $3\times3$ tetapi dalam jumlah banyak.
    * **ResNet (2015):** Memenangkan tantangan ILSVRC 2015 dengan CNN yang sangat dalam (hingga 152 lapisan). Kunci untuk melatih jaringan sedalam itu adalah *skip connections* (atau *shortcut connections*), yang memungkinkan sinyal dengan mudah melewati seluruh jaringan. Ini memaksa jaringan untuk memodelkan residual $f(x)=h(x)-x$ daripada fungsi target $h(x)$, yang dapat mempercepat pelatihan jika fungsi target mendekati fungsi identitas.
    * **SENet (2017):** Memenangkan tantangan ILSVRC 2017. Memperluas arsitektur yang ada dengan menambahkan *SE block* ke setiap unit (modul *inception* atau unit *residual*). *SE block* menganalisis *output* unit, berfokus pada dimensi kedalaman, dan mempelajari fitur mana yang biasanya paling aktif bersama. Kemudian menggunakan informasi ini untuk merekalibrasi *feature map*.
    * **Xception (Extreme Inception):** Diajukan oleh François Chollet pada 2016, secara signifikan mengungguli Inception-v3. Menggabungkan ide GoogLeNet dan ResNet, tetapi mengganti modul *inception* dengan lapisan *depthwise separable convolution* (atau *separable convolution*). Lapisan ini mengasumsikan pola spasial dan *cross-channel* dapat dimodelkan secara terpisah.
* **Transfer Learning dengan Model Pretrained:** Seringkali lebih baik menggunakan lapisan bawah dari model yang sudah dilatih (pretrained) jika data pelatihan tidak cukup. `keras.applications` menyediakan model-model ini. Gambar perlu diubah ukurannya dan diproses sebelumnya menggunakan fungsi `preprocess_input()` yang disediakan oleh model. Lapisan model pretrained biasanya dibekukan pada awal pelatihan untuk menghindari kerusakan bobot yang sudah terlatih, kemudian dapat dicairkan dan dilanjutkan pelatihan dengan *learning rate* yang lebih rendah.
* **Classification and Localization:** Melokalisasi objek dapat dinyatakan sebagai tugas regresi, memprediksi empat angka untuk *bounding box* (koordinat pusat, tinggi, dan lebar). Ini melibatkan penambahan lapisan *output dense* kedua dengan empat unit dan melatihnya menggunakan *MSE loss*. Metrik yang umum digunakan adalah Intersection over Union (IoU).
* **Object Detection:** Tugas mengklasifikasikan dan melokalisasi beberapa objek dalam gambar. Pendekatan lama melibatkan *sliding* CNN di seluruh gambar, tetapi ini lambat. *Non-max suppression* digunakan untuk menghilangkan *bounding box* yang tidak perlu.
* **Fully Convolutional Networks (FCN):** Mengganti lapisan *dense* di bagian atas CNN dengan lapisan konvolusional. FCN dapat memproses gambar dengan ukuran berapa pun dan lebih efisien daripada *sliding window* CNN tradisional.
* **You Only Look Once (YOLO):** Arsitektur deteksi objek yang sangat cepat dan akurat. YOLOv3 menghasilkan lima *bounding box* per *grid cell*, masing-masing dengan skor *objectness* dan probabilitas kelas. Ini memprediksi *offset* relatif terhadap koordinat *grid cell* dan menggunakan fungsi aktivasi logistik. Ini juga menggunakan *anchor boxes* (dimensi *bounding box* representatif yang ditemukan dengan K-Means) dan dilatih dengan gambar dari skala yang berbeda.
* **Mean Average Precision (mAP):** Metrik umum dalam tugas deteksi objek. AP (Average Precision) adalah rata-rata presisi maksimum yang dapat dicapai pada berbagai tingkat *recall*. mAP (mean Average Precision) adalah rata-rata AP di semua kelas.
* **Semantic Segmentation:** Setiap piksel diklasifikasikan berdasarkan kelas objek yang dimilikinya. Kesulitan utamanya adalah CNN kehilangan resolusi spasial karena lapisan dengan *stride* lebih besar dari 1. Solusi melibatkan penggunaan lapisan *transposed convolutional* (atau *deconvolutional*) dan *skip connections* untuk memulihkan resolusi spasial yang hilang selama operasi konvolusi dan *pooling*.
* **Instance Segmentation:** Mirip dengan segmentasi semantik, tetapi membedakan setiap *instance* objek dari kelas yang sama (misalnya, membedakan setiap sepeda di gambar). Mask R-CNN adalah arsitektur populer untuk tugas ini.

**Penjelasan Teoritis:**

CNN adalah jenis jaringan saraf tiruan yang dirancang khusus untuk memproses data visual, mengambil inspirasi dari cara kerja korteks visual mamalia.

1.  **Receptive Fields Lokal dan Pembagian Bobot (Local Receptive Fields and Weight Sharing):**
    * **Teori:** Tidak seperti Jaringan Saraf Dalam (DNN) *fully connected* di mana setiap neuron di satu lapisan terhubung ke setiap neuron di lapisan sebelumnya, neuron di lapisan konvolusional hanya terhubung ke sub-region kecil dari lapisan masukan, yang disebut *receptive field* lokal. Selain itu, semua neuron dalam *feature map* yang sama berbagi bobot dan *bias* yang sama. Ini berarti bahwa filter yang sama digeser (*slide*) ke seluruh gambar untuk mendeteksi pola yang sama di berbagai lokasi.
    * **Implikasi:**
        * **Deteksi Pola Lokal:** Memungkinkan jaringan untuk mendeteksi pola lokal kecil (misalnya, tepi, sudut) pada tahap awal.
        * **Translation Invariance (Sebagian):** Karena bobot dibagikan, sekali CNN belajar mengenali pola di satu lokasi, ia dapat mengenalinya di lokasi lain tanpa harus mempelajarinya lagi secara spesifik untuk setiap posisi.
        * **Pengurangan Parameter:** Pembagian bobot secara drastis mengurangi jumlah parameter model, yang membantu mengatasi masalah *overfitting* dan membuat pelatihan menjadi lebih layak secara komputasi untuk gambar besar.

2.  **Hierarki Fitur (Hierarchical Feature Learning):**
    * **Teori:** CNN secara otomatis mempelajari hierarki fitur. Lapisan awal mendeteksi fitur tingkat rendah (misalnya, garis horizontal/vertikal, lingkaran). Lapisan berikutnya kemudian membangun di atas fitur-fitur ini, menggabungkannya menjadi pola yang lebih kompleks (misalnya, mata, hidung, mulut). Lapisan yang lebih dalam terus menggabungkan fitur-fitur ini menjadi representasi yang semakin abstrak (misalnya, wajah, objek lengkap).
    * **Implikasi:** Ini mencerminkan cara manusia mengenali objek dan sangat efektif untuk data visual karena objek di dunia nyata sering kali terdiri dari kombinasi fitur yang lebih kecil dan sederhana.

3.  **Lapisan Pooling (Pooling Layers):**
    * **Teori:** Lapisan *pooling* (paling sering *max pooling*) mengurangi dimensi spasial (*height* dan *width*) dari *feature map*. Ini dilakukan dengan mengambil nilai maksimum (untuk *max pooling*) atau rata-rata (untuk *average pooling*) dalam *receptive field* kecil.
    * **Implikasi:**
        * **Dimensionality Reduction:** Mengurangi jumlah parameter, penggunaan memori, dan beban komputasi, yang membantu mengontrol *overfitting*.
        * **Translation Invariance:** Karena *pooling* memilih nilai yang paling dominan dalam suatu wilayah, sedikit pergeseran pada masukan mungkin tidak mengubah *output* dari lapisan *pooling*. Ini memberikan tingkat invarian terhadap translasi kecil.
        * **Robustness:** Membuat model lebih tangguh terhadap variasi posisi objek dalam gambar.

4.  **Arsitektur Modern dan Inovasi:**
    * **ReLU Activation Function:** Digunakan secara luas karena efisiensi komputasinya dan kemampuannya untuk mengatasi masalah *vanishing gradients* dibandingkan dengan fungsi aktivasi seperti `sigmoid` atau `tanh`.
    * **Dropout:** Teknik regularisasi yang digunakan untuk mengurangi *overfitting* dengan secara acak mematikan beberapa neuron selama pelatihan.
    * **Batch Normalization (BN):** Menormalkan masukan setiap lapisan. Ini mengatasi masalah *internal covariate shift* (perubahan distribusi masukan setiap lapisan selama pelatihan), yang mempercepat pelatihan dan meningkatkan stabilitas model.
    * **Skip Connections (ResNet):** Memungkinkan *gradient* mengalir langsung melalui jaringan yang sangat dalam, mengatasi masalah *vanishing gradients* dan memungkinkan pelatihan jaringan dengan ratusan lapisan. Jaringan belajar "residual" (perbedaan antara masukan dan *output* yang diinginkan) daripada seluruh pemetaan.
    * **Inception Modules (GoogLeNet):** Mengizinkan jaringan untuk belajar pola pada skala yang berbeda secara bersamaan dengan memproses masukan secara paralel melalui berbagai filter dan *pooling* layer, kemudian menggabungkan hasilnya. Ini meningkatkan efisiensi parameter.
    * **Depthwise Separable Convolutions (Xception):** Memisahkan operasi konvolusi spasial dan *cross-channel*, mengurangi jumlah parameter dan komputasi, dan sering kali memberikan kinerja yang lebih baik.
    * **SE Blocks (SENet):** Meningkatkan kinerja dengan "merekalibrasi" *feature map*. *SE block* mempelajari pentingnya setiap *feature map* dan menyesuaikan skalanya, memungkinkan jaringan untuk lebih fokus pada fitur yang relevan.

5.  **Transfer Learning:**
    * **Teori:** Lapisan awal CNN cenderung belajar fitur umum (tepi, tekstur) yang relevan untuk berbagai tugas visi komputer, sementara lapisan yang lebih dalam belajar fitur yang lebih spesifik untuk tugas tertentu (misalnya, wajah, mobil).
    * **Implikasi:** Dengan menggunakan model yang sudah dilatih (pretrained) pada *dataset* yang besar (seperti ImageNet) dan hanya melatih ulang lapisan atas atau sedikit melakukan *fine-tuning* pada seluruh jaringan dengan *dataset* baru yang lebih kecil, kita dapat mencapai kinerja yang sangat baik dengan lebih sedikit data dan waktu pelatihan.

6.  **Tugas Visi Komputer Lanjutan:**
    * **Object Detection:** Mengklasifikasikan objek dan melokalisasinya dengan menggambar *bounding box* di sekitarnya. Pendekatan modern seperti YOLO menggunakan FCN untuk memproses seluruh gambar sekaligus dan memprediksi banyak *bounding box* dan probabilitas kelas secara efisien.
    * **Semantic Segmentation:** Mengklasifikasikan setiap piksel dalam gambar ke kelas objeknya. Ini sering kali melibatkan penggunaan lapisan *transposed convolutional* (atau *deconvolutional*) dan *skip connections* untuk memulihkan resolusi spasial yang hilang selama operasi konvolusi dan *pooling*.
    * **Instance Segmentation:** Mirip dengan segmentasi semantik, tetapi membedakan setiap *instance* objek dari kelas yang sama (misalnya, membedakan setiap sepeda di gambar). Mask R-CNN adalah arsitektur populer untuk tugas ini.

Secara keseluruhan, CNN telah merevolusi visi komputer dengan kemampuannya untuk secara otomatis mempelajari representasi hierarkis dari data gambar, yang didukung oleh inovasi arsitektural dan teknik pelatihan yang efisien.